In [1]:
import pandas as pd
import geopandas as gp 
import json
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(12,8)})
%matplotlib inline
from tqdm import tqdm
import glob
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import linregress

In [2]:
# files = glob.glob('../BL_Work/openPrescribe/serialized/*.gz')
files = glob.glob('../BL_Work/openPrescribe/savings_Impact/*.h5')
print files

['../BL_Work/openPrescribe/savings_Impact/201706.h5', '../BL_Work/openPrescribe/savings_Impact/201608.h5', '../BL_Work/openPrescribe/savings_Impact/201703.h5', '../BL_Work/openPrescribe/savings_Impact/201605.h5', '../BL_Work/openPrescribe/savings_Impact/201705.h5', '../BL_Work/openPrescribe/savings_Impact/201905.h5', '../BL_Work/openPrescribe/savings_Impact/201708.h5', '../BL_Work/openPrescribe/savings_Impact/201604.h5', '../BL_Work/openPrescribe/savings_Impact/201901.h5', '../BL_Work/openPrescribe/savings_Impact/201806.h5', '../BL_Work/openPrescribe/savings_Impact/201805.h5', '../BL_Work/openPrescribe/savings_Impact/201408.h5', '../BL_Work/openPrescribe/savings_Impact/201808.h5', '../BL_Work/openPrescribe/savings_Impact/201504.h5', '../BL_Work/openPrescribe/savings_Impact/201609.h5', '../BL_Work/openPrescribe/savings_Impact/201711.h5', '../BL_Work/openPrescribe/savings_Impact/201807.h5', '../BL_Work/openPrescribe/savings_Impact/201607.h5', '../BL_Work/openPrescribe/savings_Impact/2019

In [3]:
# pdp = pd.read_csv('../BL_Work/openPrescribe/savings_Impact/201601.h5',compression='gzip')
pdp = pd.read_hdf('../BL_Work/openPrescribe/savings_Impact/201601.h5',compression='gzip')
pdp.dropna()
for month in [ '201602' , '201603','201604','201605','201606','201607','201608','201609','201610','201611','201612']:
    file_path  = '../BL_Work/openPrescribe/savings_Impact/' + month + '.h5'
#     pdp2 = pd.read_csv(file_path,compression='gzip')
    pdp2 = pd.read_hdf(file_path,compression='gzip')
    pdp2.dropna()
    pdp.append(pdp2)
    

In [68]:
len(pdp)

9046629

In [4]:
# pdp_BNF = pd.read_csv('../BL_Work/openPrescribe/serialized_BNF/201601.gz',compression='gzip')
# pdp_BNF.dropna()
# for month in [ '201602' , '201603']:
#     file_path  = '../BL_Work/openPrescribe/serialized_BNF/' + month + '.gz'
#     pdp2 = pd.read_csv(file_path,compression='gzip')
#     pdp2.dropna()
#     pdp_BNF.append(pdp2)

In [5]:
pdp.head()

Unnamed: 0    0    1       2                3  \
0 51135        59964  Q44  01R  N81084  0101010C0BBAAAA   
  55454        64283  Q44  01R  N81614  0101010C0AAAAAA   
  69323        78846  Q44  02D  N81087  0101010C0AAAAAA   
  81064        91133  Q44  02E  N81020  0101010C0AAAAAA   
  104971      115040  Q44  02E  N81122  0101010C0BBAAAA   

                                                 4    5      6      7      8  \
0 51135   Alu-Cap_Cap 475mg                         1.0  38.39  35.57  336.0   
  55454   Alum Hydrox_Cap 475mg                     1.0  13.71  12.66  120.0   
  69323   Alum Hydrox_Cap 475mg                     1.0  31.99  29.66  280.0   
  81064   Alum Hydrox_Cap 475mg                     2.0  25.60  23.86  224.0   
  104971  Alu-Cap_Cap 475mg                         1.0  19.19  17.84  168.0   

          ...     15         16          17   18          19  20        21  \
0 51135   ...  475.0  0101010C0  112.000000  1.0  112.000000  BB  0.110952   
  55454   ...  475.0  0101010C0   40.000000  1.0   40.000000  AA  0.110952   
  69323   ...  475.0  0101010C0   93.333333  1.0   93.333333  AA  0.110952   
  81064   ...  475.0  0101010C0   74.666667  1.0   74.666667  AA  0.110952   
  104971  ...  475.0  0101010C0   56.000000  1.0   56.000000  BB  0.110952   

                22         23        24  
0 51135   0.143333  10.880000  1.413690  
  55454   0.143333   3.885714  3.958333  
  69323   0.143333   9.066667  1.696429  
  81064   0.143333   7.253333  2.120536  
  104971  0.143333   5.440000  2.827381  

[5 rows x 26 columns]

In [6]:
# pdp = pd.read_csv('../BL_Work/openPrescribe/serialized/201601.gz',compression='gzip')
# pdp.dropna()


In [6]:
# set(pdp['12'])

In [7]:
%store -r taxonomyDict

In [8]:
%store -r cityMap

In [9]:
disease_drugs = json.load(open("scratchpad/Disease_Drug_DrugBank.json",'rb'))

In [10]:
# pdp.head(n=1000)

In [11]:
ward_pop = pd.read_csv('scratchpad/ward_pop.csv')

/home/sagar/anaconda3/envs/env_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
ward_pop.head()

,Ward Code 1,Ward Name 1,Local Authority,All Ages,0,1,2,3,4,5,...,81,82,83,84,85,86,87,88,89,90+
0,E05000026,Abbey,Barking and Dagenham,"15,133",351,332,361,311,328,290,...,18,15,22,14,14,19,12,6,10,14
1,E05000027,Alibon,Barking and Dagenham,"10,853",178,185,205,205,196,222,...,31,32,23,22,34,20,14,20,17,59
2,E05000028,Becontree,Barking and Dagenham,"14,466",303,304,285,320,294,271,...,28,25,22,29,19,22,15,19,11,65
3,E05000029,Chadwell Heath,Barking and Dagenham,"11,140",196,176,175,177,199,168,...,60,41,44,43,33,38,35,28,22,146
4,E05000030,Eastbrook,Barking and Dagenham,"11,464",185,156,177,172,165,156,...,54,46,51,56,42,37,28,29,30,139


In [13]:
GP_META = pd.read_csv('../BL_Work/epraccur.csv',header=None)
GP_META.rename(columns={0:'BP_code',
                        1:'Name',
                        2:'Grouping',
                        3:'National_geo',
                        4:'Addr1',
                        5:'Addr2',
                        6:'Addr3',
                        7: 'Addr4',
                        8:'Addr5',
                        9:'Postcode',
                        10:'Open',
                        11:'Closed',
                        12:'Status',
                        13:'Org type code',
                        14:'Commissioner',
                        15:'Join provider',
                        16:'Left provider',
                        17:'Contact',
                        18:'Null1',
                        19:'Null2',
                        20:'Null3',
                        21:'Amended',
                        22:'Null4',
                        23:'Provider',
                        24:'Null5',
                        25:'Setting',
                        26:'Null6'}, 
                 inplace=True)

In [14]:
print("yo")

yo


In [15]:
GP_META.head()

,BP_code,Name,Grouping,National_geo,Addr1,Addr2,Addr3,Addr4,Addr5,Postcode,...,Contact,Null1,Null2,Null3,Amended,Null4,Provider,Null5,Setting,Null6
0,A81001,THE DENSHAM SURGERY,Y63,Q74,THE HEALTH CENTRE,LAWSON STREET,STOCKTON-ON-TEES,CLEVELAND,NaN,TS18 1HU,...,01642 672351,NaN,NaN,NaN,1,NaN,00K,NaN,4,NaN
1,A81002,QUEENS PARK MEDICAL CENTRE,Y63,Q74,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,NaN,TS18 2AW,...,01642 679681,NaN,NaN,NaN,1,NaN,00K,NaN,4,NaN
2,A81003,VICTORIA MEDICAL PRACTICE,Y54,Q74,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,NaN,TS26 8DB,...,01429 272945,NaN,NaN,NaN,0,NaN,00K,NaN,4,NaN
3,A81004,BLUEBELL MEDICAL CENTRE,Y63,Q74,TRIMDON AVENUE,ACKLAM,MIDDLESBROUGH,NaN,NaN,TS5 8SB,...,01642 827697,NaN,NaN,NaN,1,NaN,00M,NaN,4,NaN
4,A81005,SPRINGWOOD SURGERY,Y63,Q74,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,NaN,TS14 7DJ,...,01287 619611,NaN,NaN,NaN,1,NaN,00M,NaN,4,NaN


In [16]:
Open_filtered_GPs = {}
for index,row in GP_META.iterrows():
    if row ['Setting'] == 4 and row['Status'] =='A':
        Open_filtered_GPs[row['BP_code']] = row['Postcode'].strip()

In [17]:
len(Open_filtered_GPs)

6924

In [18]:
df_city = pd.read_csv('../BL_Work/lower_layer_super_output_area_2011_to_major_towns_and_cities_december_2015_lookup_in_england_and_wales.csv')

In [19]:
cityMap = {}
for name , group in df_city.groupby('TCITY15NM'):
        cityMap[name] = list(group['LSOA11CD'])

In [ ]:
df_city.head()

In [22]:
IMD_df = pd.read_csv('../BL_Work/File_7_ID_2015_All_ranks__deciles_and_scores_for_the_Indices_of_Deprivation__and_population_denominators.csv')

In [23]:
IMD_df.head()

,LSOA code (2011),LSOA name (2011),Local Authority District code (2013),Local Authority District name (2013),Index of Multiple Deprivation (IMD) Score,Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived),Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs),Income Score (rate),Income Rank (where 1 is most deprived),Income Decile (where 1 is most deprived 10% of LSOAs),...,Indoors Sub-domain Rank (where 1 is most deprived),Indoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Outdoors Sub-domain Score,Outdoors Sub-domain Rank (where 1 is most deprived),Outdoors Sub-domain Decile (where 1 is most deprived 10% of LSOAs),Total population: mid 2012 (excluding prisoners),Dependent Children aged 0-15: mid 2012 (excluding prisoners),Population aged 16-59: mid 2012 (excluding prisoners),Older population aged 60 and over: mid 2012 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
0,E01031349,Adur 001A,E07000223,Adur,12.389,21352,7,0.096,18992,6,...,20379,7,0.312,11318,4,1318,206,694,418,702.75
1,E01031350,Adur 001B,E07000223,Adur,28.619,8864,3,0.187,9233,3,...,16285,5,0.234,12445,4,1212,232,712,268,720.75
2,E01031351,Adur 001C,E07000223,Adur,11.713,22143,7,0.065,24539,8,...,25054,8,0.208,12820,4,1577,290,829,458,838.25
3,E01031352,Adur 001D,E07000223,Adur,16.446,17252,6,0.117,16087,5,...,24455,8,0.109,14350,5,1453,233,739,481,748.25
4,E01031370,Adur 001E,E07000223,Adur,18.265,15643,5,0.102,17918,6,...,20214,7,0.321,11202,4,1443,306,799,338,795.50


In [24]:
LSOA_survey_takers = json.load(open('scratchpad/LSOA_suvery_pop.json'))

In [25]:
LSOA_pop = {}
LSOA_IMD = {}
for index, row in IMD_df.iterrows():
    LSOA_pop[row['LSOA code (2011)']] = row['Total population: mid 2012 (excluding prisoners)']
    LSOA_IMD[row['LSOA code (2011)']] = row['Index of Multiple Deprivation (IMD) Score']

In [26]:
cityPop = {}
city_IMD = {}
city_survey_pop = {}
for k in cityMap:
    pop = 0
    surveypop = 0
    IMD = []
    for j in cityMap[k]:
        try:
            pop += LSOA_pop[j]
            surveypop += LSOA_survey_takers[j]
            IMD.append(LSOA_IMD[j])
        except:
            print("could not find LSOA",j)
    city_IMD[k] = {}
    if pop > 0:
        cityPop[k] = pop
        city_survey_pop[k] = surveypop
        city_IMD[k]['median_IMD'] = np.median(IMD)
        city_IMD[k]['mean_IMD'] = np.mean(IMD)

('could not find LSOA', 'W01001912')
('could not find LSOA', 'W01001913')
('could not find LSOA', 'W01001600')
('could not find LSOA', 'W01001651')
('could not find LSOA', 'W01001601')
('could not find LSOA', 'W01001652')
('could not find LSOA', 'W01001602')
('could not find LSOA', 'W01001653')
('could not find LSOA', 'W01001654')
('could not find LSOA', 'W01001603')
('could not find LSOA', 'W01001604')
('could not find LSOA', 'W01001655')
('could not find LSOA', 'W01001605')
('could not find LSOA', 'W01001606')
('could not find LSOA', 'W01001607')
('could not find LSOA', 'W01001659')
('could not find LSOA', 'W01001608')
('could not find LSOA', 'W01001660')
('could not find LSOA', 'W01001609')
('could not find LSOA', 'W01001661')
('could not find LSOA', 'W01001610')
('could not find LSOA', 'W01001662')
('could not find LSOA', 'W01001611')
('could not find LSOA', 'W01001663')
('could not find LSOA', 'W01001612')
('could not find LSOA', 'W01001664')
('could not find LSOA', 'W01001613')
(

In [27]:
# json.dump(city_survey_pop,open('city_SurveyPop.json','wb'))
# json.dump(LSOA_pop,open('LSOA_Pop.json','wb'))

In [28]:
# json.dump(city_IMD,open('city_IMD.json','wb'))

In [29]:
%store Open_filtered_GPs

Stored 'Open_filtered_GPs' (dict)


In [30]:
pdp = pdp[pdp['2'].isin(Open_filtered_GPs.keys())]

In [31]:
# pdp_BNF = pdp_BNF[pdp_BNF['2'].isin(Open_filtered_GPs.keys())]

In [32]:
population = {}
for index, row in ward_pop.iterrows():
    population[row['Ward Code 1']] = float(row['All Ages'].replace(',',''))
    

In [33]:
ward_pop.head()

,Ward Code 1,Ward Name 1,Local Authority,All Ages,0,1,2,3,4,5,...,81,82,83,84,85,86,87,88,89,90+
0,E05000026,Abbey,Barking and Dagenham,"15,133",351,332,361,311,328,290,...,18,15,22,14,14,19,12,6,10,14
1,E05000027,Alibon,Barking and Dagenham,"10,853",178,185,205,205,196,222,...,31,32,23,22,34,20,14,20,17,59
2,E05000028,Becontree,Barking and Dagenham,"14,466",303,304,285,320,294,271,...,28,25,22,29,19,22,15,19,11,65
3,E05000029,Chadwell Heath,Barking and Dagenham,"11,140",196,176,175,177,199,168,...,60,41,44,43,33,38,35,28,22,146
4,E05000030,Eastbrook,Barking and Dagenham,"11,464",185,156,177,172,165,156,...,54,46,51,56,42,37,28,29,30,139


In [34]:
population_borough = {}
for name,group in ward_pop.groupby('Local Authority'):
    population_borough[name] = 0.0
    for index , row in group.iterrows():
        population_borough[name] += float(row['All Ages'].replace(',',''))

In [35]:
sum(population_borough.values())

58381217.0

In [36]:
# compareKeys = set(ward_pop_prev.keys()).intersection(ward_luca_predict.keys())

# pearsonr([ ward_gt[k] for k in compareKeys],[ward_pop_prev[k] for k in compareKeys])

# pearsonr([ ward_gt[k] for k in compareKeys],[ward_pop_prev_BNF[k] for k in compareKeys])

# pearsonr([ ward_gt[k] for k in compareKeys],[ward_luca_predict[k] for k in compareKeys])

# sns.regplot([ ward_gt[k] for k in compareKeys],[ward_pop_prev_BNF[k] for k in compareKeys])

# sns.regplot([ ward_gt[k] for k in compareKeys],[ward_pop_prev[k] for k in compareKeys])

# sns.regplot([ ward_gt[k] for k in compareKeys],[ward_luca_predict[k] for k in compareKeys])

# sns.regplot([ ward_pop_prev[k] for k in compareKeys],[ward_luca_predict[k] for k in compareKeys])
# ax = plt.gca()
# ax.set_title("Comparison between prevalences acquired by BNF and Drugbank drug lists")
# ax.set_xlabel("Drugbank prevalence")
# ax.set_ylabel("BNF prevalence")

# pearsonr([ ward_pop_prev[k] for k in compareKeys],[ward_luca_predict[k] for k in compareKeys])

In [37]:
len(set(ward_pop['Local Authority']))

348

In [38]:
GP_postcodes_long = {}
for name , group in pdp.groupby('2'):
    postcode = list(set(group['10']))
    GP_postcodes_long[name] = str(postcode[0]).strip()

In [39]:
ward_df = pd.read_csv('../BL_Work/pcd11_par11_wd11_lad11_ew_lu.csv')

ward_df.head()

ward_postcode_map = {}
for name, group in ward_df.groupby('wd11cd'):
    ward_postcode_map[name] = list(group['pcds'])

/home/sagar/anaconda3/envs/env_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
# ward_postcode_map

In [41]:
# ward_gt

In [42]:
# ward_postcode_map

In [43]:
%store -r ward_patients_filtered

In [45]:
LSOA_dist = json.load(open('scratchpad/GP_LSOA_PATIENTSDIST.json','rb'))

In [46]:
sum(ward_patients_filtered.values())

4672559

In [47]:
disease = list(set(pdp[pd.notnull(pdp['12'])]['12']))
print disease

['psoriasis', "Meniere's disease", "crohn's disease", 'gastroparesis', 'irritable bowel syndrome', 'parkinson', 'sleep apnea', 'interstitial cystitis', 'bipolar', 'hypothyroidism', 'dementia', 'rheumatoid', 'depression', 'diabetes']


In [48]:
def func_ome(df,drugBNF,ome_map):
    df['presc_ome'] = df['8'] *df['15']*ome_map[drugBNF]
    return df

def calculateOME(pdp,ome_map):
    pdp['presc_ome'] = 0.0
    return pdp.groupby('3',as_index=False).apply(lambda df: func_ome(df , df.name, ome_map ))

In [49]:
%store -r ome_map

In [52]:
pdp_ome = pdp[pdp['3'].isin(ome_map.keys())]

In [56]:
pdp_ome = calculateOME(pdp_ome , ome_map) 

/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [57]:
len(pdp_ome)

265195

In [58]:
pdp_ome.head()

Unnamed: 0    0    1       2                3  \
494 168748       182864  Q44  12F  N85008  040702020BBABAB   
    1358838     1511460  Q48  01V  N84021  040702020BBAAAA   
    1696351     1883688  Q49  01H  A82037  040702020BBAAAA   
    2172098     2429047  Q51  02Q  C84008  040702020AAABAB   
    2212315     2473273  Q51  02X  C86022  040702020BBABAB   

                                                    4    5       6       7  \
494 168748   Cyclimorph 15_Inj 1ml Amp                 4.0  109.44  101.13   
    1358838  Cyclimorph 10_Inj 1ml Amp                 1.0   43.85   40.51   
    1696351  Cyclimorph 10_Inj 1ml Amp                 2.0   31.58   29.40   
    2172098  Morph/Cyclizine_Inj 15mg/50mg 1ml Amp     3.0    5.46    5.04   
    2212315  Cyclimorph 15_Inj 1ml Amp                 2.0   27.36   25.30   

                8  ...         16    17   18    19  20    21    22     23  \
494 168748   60.0  ...  040702020  60.0  1.5  90.0  BB  1.00  1.68  40.80   
    1358838  25.0  ...  040702020  25.0  1.0  25.0  BB  1.00  1.62  15.50   
    1696351  18.0  ...  040702020  18.0  1.0  18.0  BB  1.00  1.62  11.16   
    2172098   3.0  ...  040702020   3.0  5.0  15.0  AA  1.62  1.00   0.00   
    2212315  15.0  ...  040702020  15.0  1.5  22.5  BB  1.00  1.68  10.20   

                    24  presc_ome  
494 168748    0.250000     1800.0  
    1358838   0.400000      500.0  
    1696351   0.555556      360.0  
    2172098  16.666667      300.0  
    2212315   1.000000      450.0  

[5 rows x 27 columns]

In [60]:
All_drug_count = {}
for name, group in tqdm(pdp_ome.groupby('2')):
    all_drugs = np.sum(group['presc_ome'])
    if name in LSOA_dist:
        for k in LSOA_dist[name]:
            if k not in All_drug_count:
                All_drug_count[k] = 0.0
            All_drug_count[k]+= float(all_drugs)*float(LSOA_dist[name][k])

100%|██████████| 6877/6877 [00:04<00:00, 1626.22it/s]


In [ ]:
# All_drug_count.keys()

In [61]:
# %store All_drug_count

UsageError: Unknown variable 'All_drug_count_ome'


In [62]:
# LSOA_disease_drug_count = {}
# LSOA_disease_dosage_count = {}
# for d in tqdm(disease):
# # for d in ['dementia','rheumatoid','dementia','depression','diabetes']:
#     if d not in LSOA_disease_drug_count:
#         LSOA_disease_drug_count[d]={}
#         LSOA_disease_dosage_count[d] = {}
#     for name, group in pdp.groupby('2'):
#         if name in LSOA_dist:
#             disease_presc = group[group['12']==d]
#             disease_count = np.sum(disease_presc['5'])
#             disease_dosage_count = np.sum(disease_presc['19'])
#             for k in LSOA_dist[name]:
#                 if k not in LSOA_disease_drug_count[d]:
#                     LSOA_disease_drug_count[d][k] = 0.0
#                 if k not in LSOA_disease_dosage_count[d]:
#                     LSOA_disease_dosage_count[d][k] = 0.0
#                 LSOA_disease_dosage_count[d][k]+= float(disease_dosage_count)*float(LSOA_dist[name][k])
#                 LSOA_disease_drug_count[d][k]+= float(disease_count)*float(LSOA_dist[name][k])

In [63]:
# symptom = list(set(pdp[pd.notnull(pdp['13'])]['13']))
# print symptom

In [64]:
# LSOA_symptom_drug_count = {}
# LSOA_symptom_dosage_count = {}
# for d in tqdm(symptom):
# # for d in ['dementia','rheumatoid','dementia','depression','diabetes']:
#     if d not in LSOA_symptom_drug_count:
#         LSOA_symptom_drug_count[d]={}
#         LSOA_symptom_dosage_count[d] = {}
#     for name, group in pdp.groupby('2'):
#         if name in LSOA_dist:
#             disease_presc = group[group['13']==d]
#             disease_count = np.sum(disease_presc['5'])
#             dosage_count = np.sum(disease_presc['19'])
#             for k in LSOA_dist[name]:
#                 if k not in LSOA_symptom_drug_count[d]:
#                     LSOA_symptom_drug_count[d][k] = 0.0
#                 if k not in LSOA_symptom_dosage_count[d]:
#                     LSOA_symptom_dosage_count[d][k] = 0.0
#                 LSOA_symptom_drug_count[d][k]+= float(disease_count)*float(LSOA_dist[name][k])
#                 LSOA_symptom_dosage_count[d][k]+= float(dosage_count)*float(LSOA_dist[name][k])

In [65]:
# LSOA_disease_drug_count_BNF = {}
# for d in tqdm(disease):
# # for d in ['dementia']:
#     if d not in LSOA_disease_drug_count_BNF:
#         LSOA_disease_drug_count_BNF[d]={}
#     for name, group in tqdm(pdp_BNF.groupby('2')):
#         if name in LSOA_dist:
#             disease_presc = group[group['12']==d]
#             disease_count = np.sum(disease_presc['5'])
#             for k in LSOA_dist[name]:
#                 if k not in LSOA_disease_drug_count_BNF[d]:
#                     LSOA_disease_drug_count_BNF[d][k] = 0.0
#                 LSOA_disease_drug_count_BNF[d][k]+= float(disease_count)*float(LSOA_dist[name][k])

In [66]:
# LSOA_disease_drug_count_BNF

In [ ]:
%store -r Ward_prevalence
%store -r Ward_patients
%store -r DiseasePop_byWard

In [ ]:
len(Ward_patients['MH'])

In [ ]:
# with open('LSOA_disease_count.json','wb') as f:
#     json.dump(LSOA_disease_drug_count,f)

In [ ]:
# %store LSOA_disease_drug_count

In [ ]:
# ward_prev_LSOA= {}
# for k in LSOA_disease_drug_count['diabetes']:
#     if LSOA_disease_drug_count['diabetes'][k] > Ward_patients[k]:
#         print LSOA_disease_drug_count['diabetes'][k] , Ward_patients[k]
# #         continue
#     ward_prev_LSOA[k] = LSOA_disease_drug_count['diabetes'][k]/Ward_patients[k]

In [ ]:
df = pd.read_csv('../BL_Work/Lower_Layer_Super_Output_Area_2011_to_Ward_2015_Lookup_in_England_and_Wales.csv')

In [ ]:
df.head()

In [ ]:
boroughMap = {}
for name , group in df.groupby('LAD15NM'):
        boroughMap[name] = list(group['LSOA11CD'])

In [ ]:
city_prevalence_UK = {}
city_gt_prevalence_UK = {}

for k in cityMap:
    try:
        city_disease_presc = np.sum([LSOA_disease_dosage_count['dementia'][w] for w in cityMap[k] if not np.isnan(LSOA_disease_dosage_count['dementia'][w])])
#         city_patient_pop = np.sum([Ward_patients['DEM'][w] for w in cityMap[k] if not np.isnan(Ward_patients['DEM'][w])])
        city_disease_pop = np.sum([DiseasePop_byWard['DEM'][w] for w in cityMap[k] if not np.isnan(DiseasePop_byWard['DEM'][w])])
        city_prevalence_UK[k] = float(city_disease_presc)/float(cityPop[k])
        city_gt_prevalence_UK[k] = float(city_disease_pop)/float(cityPop[k])
    except:
        print "Some ward had wrong data"

In [ ]:
len(city_prevalence_UK)

In [ ]:
city_prevalence_UK

In [ ]:
sns.distplot(city_gt_prevalence_UK.values())

In [ ]:
sns.distplot(city_prevalence_UK.values())

In [ ]:
commonWards_UK_borough = set(city_prevalence_UK.keys()).intersection(city_gt_prevalence_UK.keys())

In [ ]:
sns.regplot([ city_prevalence_UK[k] for k in commonWards_UK_borough],[city_gt_prevalence_UK[k] for k in commonWards_UK_borough])
ax = plt.gca()
ax.set_title("Comparison between prevalences acquired by BNF and Drugbank drug lists")
ax.set_xlabel("Drugbank prevalence")
ax.set_ylabel("BNF prevalence")

In [ ]:
pearsonr([ city_prevalence_UK[k] for k in commonWards_UK_borough],[city_gt_prevalence_UK[k] for k in commonWards_UK_borough])

In [ ]:
LSOA_disease_drug_count.keys()

In [ ]:
city_prevalence_UK = {}
city_disease_prescription_count = {}
city_all_prescription_count = {}
for d in LSOA_disease_drug_count:
    if d not in city_prevalence_UK:
        city_prevalence_UK[d] = {}
        city_disease_prescription_count[d] = {}

        for k in cityMap:
            try:
                city_disease_presc = np.sum([LSOA_disease_drug_count[d][w] for w in cityMap[k] if not np.isnan(LSOA_disease_drug_count[d][w])])
                if k not in city_all_prescription_count:
                    city_all_presc = np.sum([All_drug_count[w] for w in cityMap[k] if not np.isnan(All_drug_count[w])])
                    city_all_prescription_count[k] = city_all_presc
                city_disease_prescription_count[d][k] = city_disease_presc
                city_prevalence_UK[d][k] = float(city_disease_presc)/float(cityPop[k])
            except:
                print "Some ward had wrong data"

In [ ]:
city_prevalence_UK.keys()

In [ ]:
sum(city_all_prescription_count.values())

In [ ]:
# borough_disease_prescription_count['diabetes']

In [ ]:
# borough_prevalence_UK['rheumatoid']

In [ ]:
FinalDisease_prevalence = {'2016Q1':city_prevalence_UK}
with open('Disease_Prevalence_City.json','wb') as f:
    json.dump(FinalDisease_prevalence,f)

In [ ]:
city_prevalence_UK

In [ ]:
symptomList= pd.read_csv("../BL_Work/Dis_sym_edited.csv")
symptomList.head()
normId = {}
for index , row in symptomList.iterrows():
    normId[row['norm_UID'].strip().lower()] = row['UID']

In [ ]:
city_prevalence_Symptom_UK = {}
city_symptom_prescription_count = {}
for d in LSOA_symptom_drug_count:
    if d not in city_prevalence_Symptom_UK:
        city_symptom_prescription_count[d] = {}
        city_prevalence_Symptom_UK[d] = {}
        city_prevalence_Symptom_UK[d]['Id'] = normId[d]
        city_prevalence_Symptom_UK[d]['prev']={}

        for k in cityMap:
            try:
                city_disease_presc = np.sum([LSOA_symptom_dosage_count[d][w] for w in cityMap[k] if not np.isnan(LSOA_symptom_drug_count[d][w])])
                city_symptom_prescription_count[d][k] = city_disease_presc
                city_prevalence_Symptom_UK[d]['prev'][k] = float(city_disease_presc)/float(cityPop[k])
            except:
                print "Some ward had wrong data"

In [ ]:
len(city_prevalence_Symptom_UK['anxiety']['prev'])

In [ ]:
# FinalSymptom_prevalence = {'2016Q1':city_prevalence_Symptom_UK}
# with open('Symptom_Prevalence_city.json','wb') as f:
#     json.dump(FinalSymptom_prevalence,f)

In [ ]:
# confidence_disease = json.load(open('../processed/DiseaseConfidence_V2_352.json','rb'))
# confidence_symptom = json.load(open('../processed/SymtomConfidence_with_normID_349.json','rb'))

In [ ]:

# DiseasePrevalence = FinalDisease_prevalence['2016Q1']
# SymptomPrevalence = FinalSymptom_prevalence['2016Q1']

In [ ]:
DiseasePop_byWard.keys()

In [ ]:
city_prevalence_UK = {}
city_prevalence_dosage_UK = {}
city_gt_prevalence_UK = {}
city_prescription_count = {}
disease_key = 'depression'
survey_key = 'DEP'
for k in tqdm(cityMap):
    try:
        city_disease_presc = np.sum([LSOA_disease_drug_count[disease_key][w] for w in cityMap[k] if not np.isnan(LSOA_disease_drug_count[disease_key][w])])
        city_disease_dosage_presc = np.sum([LSOA_disease_dosage_count[disease_key][w] for w in cityMap[k] if not np.isnan(LSOA_disease_dosage_count[disease_key][w])])
        city_patient_pop = np.sum([Ward_patients[survey_key][w] for w in cityMap[k] if not np.isnan(Ward_patients[survey_key][w])])
        city_disease_pop = np.sum([DiseasePop_byWard[survey_key][w] for w in cityMap[k] if not np.isnan(DiseasePop_byWard[survey_key][w])])
        
        city_prescription_count[k] = city_disease_presc
        city_prevalence_UK[k] = float(city_disease_presc)/float(cityPop[k])
        city_prevalence_dosage_UK[k] = float(city_disease_dosage_presc)/float(cityPop[k])
        city_gt_prevalence_UK[k] = float(city_disease_pop)/float(cityPop[k])
#         city_prevalence_UK[k] = float(city_disease_presc)/city_patient_pop
#         city_prevalence_dosage_UK[k] = float(city_disease_dosage_presc)/city_patient_pop
#         city_gt_prevalence_UK[k] = float(city_disease_pop)/city_patient_pop
    except:
        print("Some ward had wrong data")

In [ ]:
# city_prescription_count

In [ ]:
# json.dump(city_gt_prevalence_UK,open('dementia_survey_gt.json','wb'))

In [ ]:
commonWards_UK_borough = set(city_prevalence_UK.keys()).intersection(city_gt_prevalence_UK.keys())

In [ ]:
len(commonWards_UK_borough)

In [ ]:
city_gt_prevalence_UK

In [ ]:
city_prevalence_UK

In [ ]:
# commonWards_UK_borough = [k for k in commonWards_UK_borough if k != 'Dudley']

In [ ]:
# sns.regplot([ DiseasePrevalence['depression'][k]/100 for k in commonWards_UK_borough],[London_gt[k] for k in commonWards_UK_borough])
# ax = plt.gca()
# ax.set_title("Comparison between prevalences acquired by BNF and Drugbank drug lists")
# ax.set_xlabel("Drugbank prevalence")
# ax.set_ylabel("BNF prevalence")

In [ ]:
# pearsonr([ DiseasePrevalence['depression'][k]  for k in commonWards_UK_borough],[London_gt[k] for k in commonWards_UK_borough])

In [ ]:
# sns.regplot([ DiseasePrevalence['dementia'][k] for k in commonWards_UK_borough],[borough_gt_prevalence_UK[k] for k in commonWards_UK_borough])
# ax = plt.gca()
# ax.set_title("Comparison between prevalences acquired by BNF and Drugbank drug lists")
# ax.set_xlabel("Drugbank prevalence")
# ax.set_ylabel("BNF prevalence")

In [ ]:
pearsonr([ city_prevalence_UK[k] for k in commonWards_UK_borough],[city_gt_prevalence_UK[k] for k in commonWards_UK_borough])

In [ ]:
pearsonr([ city_prevalence_dosage_UK[k] for k in commonWards_UK_borough],[city_gt_prevalence_UK[k] for k in commonWards_UK_borough])

In [ ]:
def label_point(x, y, val, ax ):
    a = pd.concat({'x': pd.Series(x), 'y':  pd.Series(y), 'val': pd.Series(val)}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'], point['y'], str(point['val']),fontsize=12)

def filterHighestResiduals(gt,predict,names , topK):
    slope, intercept, r_value, p_value, std_err = linregress(predict,gt)
    predicted = [(slope*k)+intercept for k in gt]
    residuals = [abs(v1 - v2) for (v1,v2) in zip(predicted,predict)]
    topKidx = np.argsort(residuals)[-topK:][::-1]
    return [gt[i] for i in topKidx] , [predict[i] for i in topKidx], [names[i] for i in topKidx]

predict = []
predict_dose = []
gt = []
names = []
logpop = []
logdosage = [] 
for k in commonWards_UK_borough:
    predict.append(city_prevalence_UK[k]*1000.0)
    predict_dose.append(city_prevalence_dosage_UK[k]*10)#*1000.0)
    names.append(k)
    gt.append(city_gt_prevalence_UK[k]*1000.0)
    logpop.append(np.log(cityPop[k]))
    logdosage.append(np.log(city_prescription_count[k]))

# predict

corr1 = pearsonr(predict,gt)
corr2 = pearsonr(predict_dose,gt)
print corr1 , corr2

In [ ]:
disease_string = 'Dementia'

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.regplot(x=gt , y = predict)
ax = plt.gca()
# gt_ , predict_ , names_ = filterHighestResiduals(gt, predict , names, 10)
label_point(gt , predict , names , ax)
ax.set_title(disease_string ,fontsize=25)
plt.xticks(fontsize = 20 )
plt.yticks(fontsize = 20 )
ax.set_xlabel("Normalized disease prevalence from NHS survey",fontsize=25)
ax.set_ylabel("Normalized prescription items",fontsize=25)
ax.annotate("r = " +  str(corr1[0])[:4], xy=(0.1, 0.8), xycoords='axes fraction',fontsize = 20)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.regplot(x=gt , y = predict_dose)
ax = plt.gca()
# gt_ , predict_ , names_ = filterHighestResiduals(gt, predict , names, 10)
label_point(gt , predict_dose , names , ax)
ax.set_title(disease_string ,fontsize=25)
plt.xticks(fontsize = 20 )
plt.yticks(fontsize = 20 )
ax.set_xlabel("Normalized disease prevalence from NHS survey",fontsize=25)
ax.set_ylabel("Normalized prescription dosage (x 100)",fontsize=25)
ax.annotate("r = " +  str(corr2[0])[:4], xy=(0.1, 0.8), xycoords='axes fraction',fontsize = 20)

In [ ]:
from scipy.stats import linregress

corr = pearsonr(predict, gt)
print corr

slope, intercept, r_value, p_value, std_err = linregress(x=predict,y=gt)

print slope,  intercept , r_value , p_value , std_err

In [ ]:
from scipy.stats import linregress

corr = pearsonr(logpop, logdosage)
print corr

slope, intercept, r_value, p_value, std_err = linregress(x=logpop,y=logdosage)

print slope,  intercept , r_value , p_value , std_err

In [ ]:
r_value**2

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
p = sns.regplot(x=logpop,y=logdosage,ax=ax)
ax.set_xlabel("ln(Population)",fontsize=25)
ax.set_ylabel("ln(Quantity of prescription items)",fontsize=25)
ax.set_title(disease_string,fontsize=25)
plt.xticks(fontsize = 20 )
plt.yticks(fontsize = 20 )
# label_point(logpop , logdosage , names , ax)
ax.annotate(r"$\beta$" +": " +  str(slope)[:4], xy=(0.1, 0.8), xycoords='axes fraction',fontsize = 20)
ax.annotate(r"$R^2$" +": " +  str(r_value**2)[:4], xy=(0.1, 0.7), xycoords='axes fraction',fontsize = 20)